# Example: Nonimpulsive Maneuvers

Consider a 1000 kg spacecraft with an ion propulsion system. These systems are capable of extremely high efficiency but generally have low thrust. This particular system has an $I_{sp}$ of 10,000 s and a thrust of 2.5 N. The spacecraft is in a 300 km circular, equatorial, orbit around Earth. At $t$ = 0, the ion propulsion system is turned on, and remains on until the spacecraft reaches GEO. How long does it take for the spacecraft to reach GEO? How many orbital revolutions does it take to get to GEO? How much mass is used during the maneuver? Compare this propellant usage to a typical, high-thrust (impulsive) maneuver with a rocket with $I_{sp}$ of 300 s.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
# %matplotlib notebook
import matplotlib.pyplot as plt

R_E = 6378  # km
mu = 3.986E5  # km**3/s**2

r_1 = R_E + 300  # km
v_1 = np.sqrt(mu / r_1)  # km/s

r_2 = 42_164  # km, GEO

r_0 = np.array((r_1, 0, 0))  # km
v_0 = np.array((0, v_1, 0))  # km
m_0 = np.array((1000))  # kg
Y_0 = np.hstack((r_0, v_0, m_0))

T = 2.5E-3  # kN
I_sp = 10_000  # s
g_0 = 9.807E-3  # km/s**2

In [ ]:
def nonimpulsive_maneuver(t, Y, mu, T, I_sp, g_0):
    r = np.sqrt(np.dot(Y[0:3], Y[0:3]))
    v = np.sqrt(np.dot(Y[3:6], Y[3:6]))
    m = Y[-1]
    dY_dt = np.zeros(len(Y))
    dY_dt[0:3] = Y[3:6]
    dY_dt[3:6] = -mu * Y[0:3] / r**3 + T * Y[3:6] / (m * v)
    dY_dt[-1] = - T / (I_sp * g_0)
    return dY_dt

def reached_destination(t, Y, mu, T, I_sp, g_0):
    r_vec = Y[0:3]
    r = np.sqrt(np.dot(r_vec, r_vec))
    return r - r_2

reached_destination.terminal = True

def mass(t, Y, mu, T, I_sp, g_0):
    return Y[-1]

mass.terminal = True

def orbit(t, Y, mu, T, I_sp, g_0):
    return Y[1]

# Only trigger when going from negative to positive
orbit.direction = 1

In [ ]:
t_end = 2_000_000  # s
t_eval = np.linspace(0, t_end, int(1E6))
sol = solve_ivp(
    nonimpulsive_maneuver,
    t_span=(0, t_end),
    y0=Y_0,
    t_eval=t_eval,
    events=(reached_destination, mass, orbit),
    rtol=1E-12,
    atol=1E-15,
    method="DOP853",
    args=(mu, T, I_sp, g_0)
)
print(sol.status)

In [ ]:
r_vec = sol.y[0:3].T
r = np.sqrt(r_vec[:, 0]**2 + r_vec[:, 1]**2 + r_vec[:, 2]**2)
v_vec = sol.y[3:6].T
v = np.sqrt(v_vec[:, 0]**2 + v_vec[:, 1]**2 + v_vec[:, 2]**2)
m = sol.y[-1]

In [ ]:
plt.rc("font", size=20)
fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect("equal")
ax.plot(r_vec[:, 0], r_vec[:, 1])
ax.plot(np.linspace(-r_2, r_2, 1000), np.sqrt(r_2**2 - np.linspace(-r_2, r_2, 1000)**2))
orbit_crossings = sol.y_events[2][:, 0]
ax.plot(orbit_crossings, np.zeros(orbit_crossings.shape), 'ro', fillstyle='none')
print("Number of orbits", f"{orbit_crossings.shape}")

In [ ]:
print(f"The flight used: {m_0 - m[-1]} kg of propellant")

In [ ]:
print(f"Time of flight: {sol.t_events[0] / (3600 * 24)} days")